# Imports and Functions

In [94]:
import pandas as pd
import numpy as np
import json
import regex as re

In [ ]:
#defender name cleaning
def fix_defender_name(name):
    if pd.isna(name):
        return None  
    if ',' in name:
        parts = name.split(', ')
        if len(parts) == 2:
            return f"{parts[1].title()} {parts[0].title()}"
    return name.title()  #capitalization done anyway

#comfort condition creation
def comfort_conditions(df):
    shot = df["above_mid_shot_dist"]
    def_dist = df["above_mid_close_def_dist"]

    #conditions
    if shot == 0 and def_dist == 0:
        return "closer_shooting_closer_defender"
    elif shot == 1 and def_dist == 0:
        return "further_shooting_closer_defender"
    elif shot == 0 and def_dist == 1:
        return "closer_shooting_further_defender"
    else:
        return "further_shooting_further_defender"

# Goals

- for each pair of players (A,B), we define the fear score of A when facing B is the hit rate, such that B is closest defender when A is shooting. Based on the fear score, for each player, please find out who is his "most unwanted defender"
- for each player, we define the comfortable zone of shooting as a matrix of {SHOT_DIST, CLOSE_DEF_DIST, SHOT_CLOCK}. Develop a MapReduce-based algorithm to classify each player's records into 4 comfortable zones. Considering the hit rate, which zone is the best for James Harden, Chris Paul, Stephen Curry, and Lebron James. 


2
- use median/mean for a metric (high- low)
- to consider a quadrant of comfort, we cannot apply shot clock directly
- however, we can make a report inclusive of shot-clock
    - use a squadrant on SHOT_DIST, CLOSE_DEF_DIST
    - then break it apart into two grouping where shot clock is low and shot clock is high

# Notes

# Data

In [99]:
df = pd.read_csv("../../data/shot_logs.csv")

In [97]:
df["SHOT_DIST"].max()

47.2

In [101]:
df.size

2689449

In [102]:
df.shape

(128069, 21)

In [107]:
df.isnull().sum()

GAME_ID                          0
MATCHUP                          0
LOCATION                         0
W                                0
FINAL_MARGIN                     0
SHOT_NUMBER                      0
PERIOD                           0
GAME_CLOCK                       0
SHOT_CLOCK                    5567
DRIBBLES                         0
TOUCH_TIME                       0
SHOT_DIST                        0
PTS_TYPE                         0
SHOT_RESULT                      0
CLOSEST_DEFENDER                 0
CLOSEST_DEFENDER_PLAYER_ID       0
CLOSE_DEF_DIST                   0
FGM                              0
PTS                              0
player_name                      0
player_id                        0
dtype: int64

In [85]:
df.head()

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,...,SHOT_DIST,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,player_name,player_id
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,...,7.7,2,made,"Anderson, Alan",101187,1.3,1,2,brian roberts,203148
1,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,2,1,0:14,3.4,0,...,28.2,3,missed,"Bogdanovic, Bojan",202711,6.1,0,0,brian roberts,203148
2,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,3,1,0:00,NaN,3,...,10.1,2,missed,"Bogdanovic, Bojan",202711,0.9,0,0,brian roberts,203148
3,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,4,2,11:47,10.3,2,...,17.2,2,missed,"Brown, Markel",203900,3.4,0,0,brian roberts,203148
4,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,5,2,10:34,10.9,2,...,3.7,2,missed,"Young, Thaddeus",201152,1.1,0,0,brian roberts,203148


In [87]:
sorted(df.columns)

['CLOSEST_DEFENDER',
 'CLOSEST_DEFENDER_PLAYER_ID',
 'CLOSE_DEF_DIST',
 'DRIBBLES',
 'FGM',
 'FINAL_MARGIN',
 'GAME_CLOCK',
 'GAME_ID',
 'LOCATION',
 'MATCHUP',
 'PERIOD',
 'PTS',
 'PTS_TYPE',
 'SHOT_CLOCK',
 'SHOT_DIST',
 'SHOT_NUMBER',
 'SHOT_RESULT',
 'TOUCH_TIME',
 'W',
 'player_id',
 'player_name']

# Through Pandas

Let's consider performing this analysis on 3 individual games among 6 different teams before scaling this information across multiple computers. 

- we have a column of closest defender and closest defender player id
    - `CLOSEST_DEFENDER`, `CLOSEST_DEFENDER_PLAYER_ID`
    - we may need to consider the `CLOSE_DEF_DIST` into consideration to be sure the closest defender is actually the closest defender compared to everyone else. 
    - on a per game basis we would need to identify all player matchings of closest defender as unique pairings
    - then calculate an aggregated fear score for the players per all time in such game
        - we do not yet begin omitting anything....as there are instances in games where maybe a stronger fear score exists for one player, but across the season 
- fear score must be looked into and likely manually calculated
    - how do we aggregate the fear score across all of the instances of the game in which all of this happens?
        - hopefully it's simply addition
        - if its average, we would need to preserve the addition and instances in which the addition is constituted per player defender pairing 

- let's first isolate down some games and full-teams. 

- need to pull out unique team names from everything...going to grab first a unique list of the matchups

In [ ]:
df = pd.read_csv("../../data/shot_logs.csv")


#defender Cleaning
df["CLOSEST_DEFENDER"] = df["CLOSEST_DEFENDER"].apply(fix_defender_name)

#player cleaning
df["player_name"] = df["player_name"].str.title()

#null handling
df.dropna(inplace=True)


##Fear Score section
#fear_score table generation
fear_scores = (
    df.groupby(["player_name", "CLOSEST_DEFENDER"])
    .agg(
        total_season_FGM_on_defender = ("FGM", "sum"),
        total_season_shotson_defender = ("FGM", "count")
    )
    .reset_index()
)
fear_scores["season_hit_rate_on_defender"] = (fear_scores["total_season_FGM_on_defender"] / fear_scores["total_season_shotson_defender"]) #actual fear score


##relevant player threshold setting
credible_df = fear_scores[
    (fear_scores["total_season_FGM_on_defender"] >= 3) #blanket check on proper pairings....some players omitted entirely perhaps while others likely don't show enough interaction on a pairing
]

#most unwanted defenders (ANSWER)
most_unwanted_defenders = (
    credible_df.sort_values(by=["player_name", "season_hit_rate_on_defender"])
    .groupby("player_name")
    .first()
    .reset_index()
)


##Comfort Zone section
df["above_mid_shot_dist"] = df["SHOT_DIST"].apply(lambda x: 1 if x >= (df["SHOT_DIST"].max()/2) else 0)
df["above_mid_close_def_dist"] = df["CLOSE_DEF_DIST"].apply(lambda x: 1 if x >= (df["CLOSE_DEF_DIST"].max()/2) else 0)
df["above_mid_shot_clock"] = df["SHOT_CLOCK"].apply(lambda x: 1 if x >= (df["SHOT_CLOCK"].max()/2) else 0)

df = df.merge(
    credible_df, 
    on=["player_name", "CLOSEST_DEFENDER"], 
    how="left"  # left join to preserve all original rows
)

df.dropna(inplace=True)

#apply comfort zones    
df['comfort_zone'] = df.apply(comfort_conditions, axis=1)


## Individual player focus
player_focus = ["James Harden", "Lebron James", "Stephen Curry", "Chris Paul"]

##shot clock above middle
tdf = df.copy()

above_clock = tdf[
    (tdf["above_mid_shot_clock"] == 1) &
    (tdf["player_name"].isin(player_focus))
]


above_clock_hit_rate = above_clock.groupby(["player_name", "comfort_zone"]).agg(
    total_fgm = ("FGM", "sum"),
    total_fga = ("FGM", "count")
).assign(
    total_hit_rate = lambda x: (x["total_fgm"] / x["total_fga"]) * 100
)



##shot clock below middle
below_clock = tdf[
    (tdf["above_mid_shot_clock"] == 0) &
    (tdf["player_name"].isin(player_focus))
]

 
below_clock_hit_rate = below_clock.groupby(['player_name', 'comfort_zone']).agg(
    total_fgm = ("FGM", "sum"),
    total_fga = ("FGM", 'count')
).assign(
    total_hit_rate = lambda x: (x["total_fgm"] / x["total_fga"]) * 100
)


In [68]:
df

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,...,PTS,player_name,player_id,above_mid_shot_dist,above_mid_close_def_dist,above_mid_shot_clock,total_season_FGM_on_defender,total_season_shotson_defender,season_hit_rate_on_defender,comfort_zone
17,21400859,"FEB 27, 2015 - CHA @ BOS",A,L,-8,5,4,10:29,20.8,0,...,3,Brian Roberts,203148,1,0,1,3.0,7.0,0.428571,further_shooting_closer_defender
41,21400768,"FEB 08, 2015 - CHA vs. IND",H,L,-1,3,1,4:22,16.5,7,...,0,Brian Roberts,203148,0,0,1,3.0,8.0,0.375000,closer_shooting_closer_defender
42,21400768,"FEB 08, 2015 - CHA vs. IND",H,L,-1,4,1,0:31,11.3,0,...,2,Brian Roberts,203148,0,0,0,3.0,8.0,0.375000,closer_shooting_closer_defender
43,21400768,"FEB 08, 2015 - CHA vs. IND",H,L,-1,5,2,11:47,9.7,1,...,0,Brian Roberts,203148,1,0,0,3.0,8.0,0.375000,further_shooting_closer_defender
45,21400768,"FEB 08, 2015 - CHA vs. IND",H,L,-1,7,3,5:19,9.4,0,...,3,Brian Roberts,203148,1,0,0,3.0,8.0,0.375000,further_shooting_closer_defender
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122491,21400033,"NOV 01, 2014 - BKN @ DET",A,W,12,2,2,10:02,7.3,0,...,0,Jarrett Jack,101127,1,0,0,3.0,7.0,0.428571,further_shooting_closer_defender
122492,21400033,"NOV 01, 2014 - BKN @ DET",A,W,12,3,4,10:57,14.3,2,...,2,Jarrett Jack,101127,0,0,1,3.0,7.0,0.428571,closer_shooting_closer_defender
122493,21400033,"NOV 01, 2014 - BKN @ DET",A,W,12,4,4,8:34,19.8,0,...,0,Jarrett Jack,101127,1,0,1,3.0,7.0,0.428571,further_shooting_closer_defender
122496,21400006,"OCT 29, 2014 - BKN @ BOS",A,L,-16,3,2,7:46,7.0,1,...,2,Jarrett Jack,101127,0,0,0,8.0,14.0,0.571429,closer_shooting_closer_defender


In [79]:
player_focus = ["James Harden", "Lebron James", "Stephen Curry", "Chris Paul"]

##shot clock above middle
tdf = df.copy()

above_clock = tdf[
    (tdf["above_mid_shot_clock"] == 1) &
    (tdf["player_name"].isin(player_focus))
]


above_clock_hit_rate = above_clock.groupby(["player_name", "comfort_zone"]).agg(
    total_fgm = ("FGM", "sum"),
    total_fga = ("FGM", "count")
).assign(
    total_hit_rate = lambda x: (x["total_fgm"] / x["total_fga"]) * 100
)



##shot clock below middle
below_clock = tdf[
    (tdf["above_mid_shot_clock"] == 0) &
    (tdf["player_name"].isin(player_focus))
]

 
below_clock_hit_rate = below_clock.groupby(['player_name', 'comfort_zone']).agg(
    total_fgm = ("FGM", "sum"),
    total_fga = ("FGM", 'count')
).assign(
    total_hit_rate = lambda x: (x["total_fgm"] / x["total_fga"]) * 100
)

In [80]:
above_clock_hit_rate

total_fgm  total_fga  \
player_name   comfort_zone                                             
Chris Paul    closer_shooting_closer_defender         111        190   
              further_shooting_closer_defender         31         66   
James Harden  closer_shooting_closer_defender          90        151   
              further_shooting_closer_defender         47        128   
Lebron James  closer_shooting_closer_defender         121        191   
              closer_shooting_further_defender          2          2   
              further_shooting_closer_defender         26         65   
Stephen Curry closer_shooting_closer_defender         142        213   
              further_shooting_closer_defender         90        205   

                                                total_hit_rate  
player_name   comfort_zone                                      
Chris Paul    closer_shooting_closer_defender        58.421053  
              further_shooting_closer_defender       46.969697  
James Harden  closer_shooting_closer_defender        59.602649  
              further_shooting_closer_defender       36.718750  
Lebron James  closer_shooting_closer_defender        63.350785  
              closer_shooting_further_defender      100.000000  
              further_shooting_closer_defender       40.000000  
Stephen Curry closer_shooting_closer_defender        66.666667  
              further_shooting_closer_defender       43.902439

In [81]:
below_clock_hit_rate

total_fgm  total_fga  \
player_name   comfort_zone                                             
Chris Paul    closer_shooting_closer_defender          74        125   
              further_shooting_closer_defender         21         47   
James Harden  closer_shooting_closer_defender          72        137   
              further_shooting_closer_defender         46         92   
Lebron James  closer_shooting_closer_defender         101        193   
              further_shooting_closer_defender         42        107   
Stephen Curry closer_shooting_closer_defender          39         62   
              further_shooting_closer_defender         39         88   

                                                total_hit_rate  
player_name   comfort_zone                                      
Chris Paul    closer_shooting_closer_defender        59.200000  
              further_shooting_closer_defender       44.680851  
James Harden  closer_shooting_closer_defender        52.554745  
              further_shooting_closer_defender       50.000000  
Lebron James  closer_shooting_closer_defender        52.331606  
              further_shooting_closer_defender       39.252336  
Stephen Curry closer_shooting_closer_defender        62.903226  
              further_shooting_closer_defender       44.318182

To do such:
- isolate down to the specific players
    - break things apart based on the 
- note the above-below boolean threshold
- calculate their 
- get their average hit rate

<!-- ### more time
##further distance, further defender
##further distance, closer defender
##closer distance, further defender
##closer distance, closer defender

### less time
##further distance, further defender
##further distance, closer defender
##closer distance, further defender
##closer distance, closer defender -->

In [66]:
def comfort_conditions(df):
    shot = df["above_mid_shot_dist"]
    def_dist = df["above_mid_close_def_dist"]

    #conditions
    if shot == 0 and def_dist == 0:
        return "closer_shooting_closer_defender"
    elif shot == 1 and def_dist == 0:
        return "further_shooting_closer_defender"
    elif shot == 0 and def_dist == 1:
        return "closer_shooting_further_defender"
    else:
        return "further_shooting_further_defender"
    
df['comfort_zone'] = df.apply(comfort_conditions, axis=1)
df

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,...,PTS,player_name,player_id,above_mid_shot_dist,above_mid_close_def_dist,above_mid_shot_clock,total_season_FGM_on_defender,total_season_shotson_defender,season_hit_rate_on_defender,comfort_zone
17,21400859,"FEB 27, 2015 - CHA @ BOS",A,L,-8,5,4,10:29,20.8,0,...,3,Brian Roberts,203148,1,0,1,3.0,7.0,0.428571,further_shooting_closer_defender
41,21400768,"FEB 08, 2015 - CHA vs. IND",H,L,-1,3,1,4:22,16.5,7,...,0,Brian Roberts,203148,0,0,1,3.0,8.0,0.375000,closer_shooting_closer_defender
42,21400768,"FEB 08, 2015 - CHA vs. IND",H,L,-1,4,1,0:31,11.3,0,...,2,Brian Roberts,203148,0,0,0,3.0,8.0,0.375000,closer_shooting_closer_defender
43,21400768,"FEB 08, 2015 - CHA vs. IND",H,L,-1,5,2,11:47,9.7,1,...,0,Brian Roberts,203148,1,0,0,3.0,8.0,0.375000,further_shooting_closer_defender
45,21400768,"FEB 08, 2015 - CHA vs. IND",H,L,-1,7,3,5:19,9.4,0,...,3,Brian Roberts,203148,1,0,0,3.0,8.0,0.375000,further_shooting_closer_defender
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122491,21400033,"NOV 01, 2014 - BKN @ DET",A,W,12,2,2,10:02,7.3,0,...,0,Jarrett Jack,101127,1,0,0,3.0,7.0,0.428571,further_shooting_closer_defender
122492,21400033,"NOV 01, 2014 - BKN @ DET",A,W,12,3,4,10:57,14.3,2,...,2,Jarrett Jack,101127,0,0,1,3.0,7.0,0.428571,closer_shooting_closer_defender
122493,21400033,"NOV 01, 2014 - BKN @ DET",A,W,12,4,4,8:34,19.8,0,...,0,Jarrett Jack,101127,1,0,1,3.0,7.0,0.428571,further_shooting_closer_defender
122496,21400006,"OCT 29, 2014 - BKN @ BOS",A,L,-16,3,2,7:46,7.0,1,...,2,Jarrett Jack,101127,0,0,0,8.0,14.0,0.571429,closer_shooting_closer_defender


In [51]:
tdf = df.merge(
    credible_df, 
    on=["player_name", "CLOSEST_DEFENDER"], 
    how="left"  # left join to preserve all original rows
)

tdf.dropna(inplace=True)

# Experiment

- how do I quantify this? break them all into two and I have this and then run twice. 

In [ ]:
tdf = df.copy()

tdf["SHOT_DIST"].max()

47.2

In [21]:
tdf["SHOT_DIST"].min()

0.0

In [28]:
tdf["CLOSE_DEF_DIST"].max()

53.2

In [23]:
tdf["CLOSE_DEF_DIST"].min()

0.0

In [24]:
tdf["SHOT_CLOCK"].max()

24.0

In [26]:
tdf["SHOT_CLOCK"].min()

0.0

In [34]:
tdf["ABOVE_MIDDLE_SHOT_DIST"] = tdf["SHOT_DIST"].apply(lambda x: 1 if x >= (tdf["SHOT_DIST"].max()/2) else 0)
tdf["ABOVE_MIDDLE_CLOSE_DEF_DIST"] = tdf["CLOSE_DEF_DIST"].apply(lambda x: 1 if x >= (tdf["CLOSE_DEF_DIST"].max()/2) else 0)
tdf["ABOVE_MIDDLE_SHOT_CLOCK"] = tdf["SHOT_CLOCK"].apply(lambda x: 1 if x >= (tdf["SHOT_CLOCK"].max()/2) else 0)

In [35]:
tdf

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,...,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,player_name,player_id,ABOVE_MIDDLE_SHOT_DIST,ABOVE_MIDDLE_CLOSE_DEF_DIST,ABOVE_MIDDLE_SHOT_CLOCK
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,...,Alan Anderson,101187,1.3,1,2,Brian Roberts,203148,0,0,0
1,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,2,1,0:14,3.4,0,...,Bojan Bogdanovic,202711,6.1,0,0,Brian Roberts,203148,1,0,0
2,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,3,1,0:00,NaN,3,...,Bojan Bogdanovic,202711,0.9,0,0,Brian Roberts,203148,0,0,0
3,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,4,2,11:47,10.3,2,...,Markel Brown,203900,3.4,0,0,Brian Roberts,203148,0,0,0
4,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,5,2,10:34,10.9,2,...,Thaddeus Young,201152,1.1,0,0,Brian Roberts,203148,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128064,21400006,"OCT 29, 2014 - BKN @ BOS",A,L,-16,5,3,1:52,18.3,5,...,Marcus Smart,203935,0.8,0,0,Jarrett Jack,101127,0,0,1
128065,21400006,"OCT 29, 2014 - BKN @ BOS",A,L,-16,6,4,11:28,19.8,4,...,Evan Turner,202323,0.6,1,2,Jarrett Jack,101127,0,0,1
128066,21400006,"OCT 29, 2014 - BKN @ BOS",A,L,-16,7,4,11:10,23.0,2,...,Marcus Thornton,201977,4.2,1,2,Jarrett Jack,101127,0,0,1
128067,21400006,"OCT 29, 2014 - BKN @ BOS",A,L,-16,8,4,2:37,9.1,4,...,Avery Bradley,202340,3.0,0,0,Jarrett Jack,101127,0,0,0


In [ ]:

### more time
more_time = tdf[
    (tdf["ABOVE_MIDDLE_SHOT_CLOCK"] == 1)
]
##further distance, further defender
##further distance, closer defender
##closer distance, further defender
##closer distance, closer defender

### less time
##further distance, further defender
##further distance, closer defender
##closer distance, further defender
##closer distance, closer defender


,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,...,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,player_name,player_id,ABOVE_MIDDLE_SHOT_DIST
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,...,2,made,Alan Anderson,101187,1.3,1,2,Brian Roberts,203148,0
1,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,2,1,0:14,3.4,0,...,3,missed,Bojan Bogdanovic,202711,6.1,0,0,Brian Roberts,203148,1
2,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,3,1,0:00,NaN,3,...,2,missed,Bojan Bogdanovic,202711,0.9,0,0,Brian Roberts,203148,0
3,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,4,2,11:47,10.3,2,...,2,missed,Markel Brown,203900,3.4,0,0,Brian Roberts,203148,0
4,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,5,2,10:34,10.9,2,...,2,missed,Thaddeus Young,201152,1.1,0,0,Brian Roberts,203148,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128064,21400006,"OCT 29, 2014 - BKN @ BOS",A,L,-16,5,3,1:52,18.3,5,...,2,missed,Marcus Smart,203935,0.8,0,0,Jarrett Jack,101127,0
128065,21400006,"OCT 29, 2014 - BKN @ BOS",A,L,-16,6,4,11:28,19.8,4,...,2,made,Evan Turner,202323,0.6,1,2,Jarrett Jack,101127,0
128066,21400006,"OCT 29, 2014 - BKN @ BOS",A,L,-16,7,4,11:10,23.0,2,...,2,made,Marcus Thornton,201977,4.2,1,2,Jarrett Jack,101127,0
128067,21400006,"OCT 29, 2014 - BKN @ BOS",A,L,-16,8,4,2:37,9.1,4,...,2,missed,Avery Bradley,202340,3.0,0,0,Jarrett Jack,101127,0
